

# Publicación tablas en Crossdata

In [ ]:
import pandas as pd
from pystratio.xd.xdsession import XDSession
from pyspark.sql import functions as F



## Creación de contexto Crossdata para poder acceder al catálogo

In [ ]:
def instantiate_xd():
    xd = XDSession(sc)

xd = XDSession(sc)

pd.set_option('display.width',500)
pd.set_option('display.max_columns',50)
pd.set_option('display.max_rows',300)
pd.set_option('display.max_colwidth',-1)



## Prerrequisitos para ejecutar este script
   * Debe subirse al sandbox mediante file explorer un fichero con nombre `bank_products_2019.csv` en la carpeta raíz.
   * Hay que cambiar todos los PATH de lectura-escritura, modificando la UUOO del sandbox correspondiente en la variable de la siguiente celda

In [ ]:
SANDBOX_NAME = 'fesp'
DATA_PATH = "/data/sandboxes/" + SANDBOX_NAME + "/data/data/"



## Creación de fichero parquet



Creación del Parquet a partir del *csv* (Solamente es necesario ejecutarlo una vez, aunque no pasa nada si se ejecuta más veces). Este ejemplo es con un fichero ingestado en el sandbox. Para leer directamente de Master, simplemente habría que cambiar el path apuntando al fichero deseado.




### Lectura del fichero csv



Este paso sólo es necesario hacerlo si el fichero que hemos subido al sandbox es un *csv*. Si el fichero que queremos leer ya está en formato *parquet*, no habría que ejecutar las dos celdas siguientes

In [ ]:
# Path del fichero csv subido al sandbox
file_path = DATA_PATH + "bank_products_2019.csv"

#leemos el fichero csv
tablasmicro = spark.read.csv(file_path,header=True,inferSchema=True, sep=',')

In [ ]:
tablasmicro.limit(5).toPandas()

,_c0,Row_ID,Order_ID,Account_Opening_Date,Product_Contracting_Date,Contracting_Mode,Customer_Name,Client,Country,City,Postal_Code,Region,Product_ID,Product,Sub-Product,Sales,Quantity,Discount,Profit,Customer_num
0,0,1,CA-2016-152156,08/11/2016,11/11/2016,Web,Claire Gute,Final Client,United States,Henderson; Kentucky,42420,South,FUR-BO-10001798,Office product,Pension Plan,"261,96",2,0,"41,9136",12520
1,1,2,CA-2016-152156,08/11/2016,11/11/2016,Web,Claire Gute,Final Client,United States,Henderson; Kentucky,42420,South,FUR-CH-10000454,Office product,Paysheets,"731,94",3,0,"219,582",12520
2,2,3,CA-2016-138688,12/06/2016,16/06/2016,Web,Darrin Van Huff,Big Accounts,United States,Los Angeles; California,90036,West,OFF-LA-10000240,Loans and mortgages,Mastercard,"14,62",2,0,"6,8714",13045
3,3,4,US-2015-108966,11/10/2015,18/10/2015,Bank Branch,Sean O'Donnell,Final Client,United States,Fort Lauderdale; Florida,33311,South,FUR-TA-10000577,Office product,Home insurance,"957,5775",5,"0,45","-383,031",20335
4,4,5,US-2015-108966,11/10/2015,18/10/2015,Bank Branch,Sean O'Donnell,Final Client,United States,Fort Lauderdale; Florida,33311,South,OFF-ST-10000760,Loans and mortgages,Others,"22,368",2,"0,2","2,5164",20335




### Estudio, transformación y creación de columnas



Estudiamos el tipo de las columnas y vemos que `Sales`, `Discount` y `Profit` son strings. Esto se debe a que los decimales están indicados con comas y Spark sólo los reconoce como puntos

In [ ]:
tablasmicro.dtypes

[('_c0', 'int'),
 ('Row_ID', 'int'),
 ('Order_ID', 'string'),
 ('Account_Opening_Date', 'string'),
 ('Product_Contracting_Date', 'string'),
 ('Contracting_Mode', 'string'),
 ('Customer_Name', 'string'),
 ('Client', 'string'),
 ('Country', 'string'),
 ('City', 'string'),
 ('Postal_Code', 'int'),
 ('Region', 'string'),
 ('Product_ID', 'string'),
 ('Product', 'string'),
 ('Sub-Product', 'string'),
 ('Sales', 'string'),
 ('Quantity', 'int'),
 ('Discount', 'string'),
 ('Profit', 'string'),
 ('Customer_num', 'int')]



Reemplazamos la "," por "," en estas columnas y hacemos casting a tipo float. Hacemos casting a tipo float también de la columna `Quantity` para evitar problemas más adelante

In [ ]:
for i in ['Sales', 'Profit', 'Discount', 'Quantity']:
    tablasmicro = tablasmicro.withColumn(i, F.regexp_replace(F.col(i), ',', '.'))
    tablasmicro = tablasmicro.withColumn(i, F.col(i).cast('float'))

In [ ]:
tablasmicro.select('Sales', 'Profit', 'Discount', 'Quantity').show(5)

+--------+--------+--------+--------+
|   Sales|  Profit|Discount|Quantity|
+--------+--------+--------+--------+
|  261.96| 41.9136|     0.0|     2.0|
|  731.94| 219.582|     0.0|     3.0|
|   14.62|  6.8714|     0.0|     2.0|
|957.5775|-383.031|    0.45|     5.0|
|  22.368|  2.5164|     0.2|     2.0|
+--------+--------+--------+--------+
only showing top 5 rows





Comprobamos que ha cambiado el tipo de variable

In [ ]:
tablasmicro.dtypes

[('_c0', 'int'),
 ('Row_ID', 'int'),
 ('Order_ID', 'string'),
 ('Account_Opening_Date', 'string'),
 ('Product_Contracting_Date', 'string'),
 ('Contracting_Mode', 'string'),
 ('Customer_Name', 'string'),
 ('Client', 'string'),
 ('Country', 'string'),
 ('City', 'string'),
 ('Postal_Code', 'int'),
 ('Region', 'string'),
 ('Product_ID', 'string'),
 ('Product', 'string'),
 ('Sub-Product', 'string'),
 ('Sales', 'float'),
 ('Quantity', 'float'),
 ('Discount', 'float'),
 ('Profit', 'float'),
 ('Customer_num', 'int')]



Cambiamos el valor de los registros donde la columna `Sub-Product` sea "Debit Card" para que la columna `Product` tome el valor "Loans and mortgages"

In [ ]:
from pyspark.sql import functions as F

tablasmicro = tablasmicro\
            .withColumn("Product", F.udf( lambda x,y : 'Loans and mortgages' if y == "Debit Card" else x)(F.col("Product"), F.col("Sub-Product")))

tablasmicro.where('`Product`=="Loans and mortgages"').select('Sub-Product').distinct().show()

+-----------+
|Sub-Product|
+-----------+
|    Prepaid|
|Credit Card|
|      Money|
| Mastercard|
|    Loyalty|
|     Others|
|  Revolving|
| Debit Card|
+-----------+



In [ ]:
tablasmicro.where('`Product`=="Cards"').select('Sub-Product').distinct().show()

+------------+
| Sub-Product|
+------------+
|Vehicle Loan|
|Express loan|
|  Study Loan|
+------------+





Vemos que la columna `City` tiene concatenado el nombre de la ciudad y el estado. Vamos a transformarla en dos columnas (una con la ciudad y otra con el estado)

In [ ]:
tablasmicro.limit(5).toPandas()

,_c0,Row_ID,Order_ID,Account_Opening_Date,Product_Contracting_Date,Contracting_Mode,Customer_Name,Client,Country,City,Postal_Code,Region,Product_ID,Product,Sub-Product,Sales,Quantity,Discount,Profit,Customer_num
0,0,1,CA-2016-152156,08/11/2016,11/11/2016,Web,Claire Gute,Final Client,United States,Henderson; Kentucky,42420,South,FUR-BO-10001798,Office product,Pension Plan,261.959991,2.0,0.00,41.913601,12520
1,1,2,CA-2016-152156,08/11/2016,11/11/2016,Web,Claire Gute,Final Client,United States,Henderson; Kentucky,42420,South,FUR-CH-10000454,Office product,Paysheets,731.940002,3.0,0.00,219.582001,12520
2,2,3,CA-2016-138688,12/06/2016,16/06/2016,Web,Darrin Van Huff,Big Accounts,United States,Los Angeles; California,90036,West,OFF-LA-10000240,Loans and mortgages,Mastercard,14.620000,2.0,0.00,6.871400,13045
3,3,4,US-2015-108966,11/10/2015,18/10/2015,Bank Branch,Sean O'Donnell,Final Client,United States,Fort Lauderdale; Florida,33311,South,FUR-TA-10000577,Office product,Home insurance,957.577515,5.0,0.45,-383.031006,20335
4,4,5,US-2015-108966,11/10/2015,18/10/2015,Bank Branch,Sean O'Donnell,Final Client,United States,Fort Lauderdale; Florida,33311,South,OFF-ST-10000760,Loans and mortgages,Others,22.368000,2.0,0.20,2.516400,20335


In [ ]:
tablasmicro = tablasmicro.withColumn("City2", F.substring_index(F.col("City"), ";", 1))
tablasmicro = tablasmicro.withColumn("State", F.substring_index(F.col("City"), ";", -1))

tablasmicro = tablasmicro.drop("City")
tablasmicro = tablasmicro.withColumnRenamed("City2", "City")

In [ ]:
tablasmicro.limit(5).toPandas()

,_c0,Row_ID,Order_ID,Account_Opening_Date,Product_Contracting_Date,Contracting_Mode,Customer_Name,Client,Country,Postal_Code,Region,Product_ID,Product,Sub-Product,Sales,Quantity,Discount,Profit,Customer_num,City,State
0,0,1,CA-2016-152156,08/11/2016,11/11/2016,Web,Claire Gute,Final Client,United States,42420,South,FUR-BO-10001798,Office product,Pension Plan,261.959991,2.0,0.00,41.913601,12520,Henderson,Kentucky
1,1,2,CA-2016-152156,08/11/2016,11/11/2016,Web,Claire Gute,Final Client,United States,42420,South,FUR-CH-10000454,Office product,Paysheets,731.940002,3.0,0.00,219.582001,12520,Henderson,Kentucky
2,2,3,CA-2016-138688,12/06/2016,16/06/2016,Web,Darrin Van Huff,Big Accounts,United States,90036,West,OFF-LA-10000240,Loans and mortgages,Mastercard,14.620000,2.0,0.00,6.871400,13045,Los Angeles,California
3,3,4,US-2015-108966,11/10/2015,18/10/2015,Bank Branch,Sean O'Donnell,Final Client,United States,33311,South,FUR-TA-10000577,Office product,Home insurance,957.577515,5.0,0.45,-383.031006,20335,Fort Lauderdale,Florida
4,4,5,US-2015-108966,11/10/2015,18/10/2015,Bank Branch,Sean O'Donnell,Final Client,United States,33311,South,OFF-ST-10000760,Loans and mortgages,Others,22.368000,2.0,0.20,2.516400,20335,Fort Lauderdale,Florida




Por otro lado, un error que detectamos es que en la columna `Region`, la palabra "centarl" está mal escrita. Vamos a sustituirla con "central"

In [ ]:
tablasmicro.select("Region").limit(20).toPandas()

,Region
0,South
1,South
2,West
3,South
4,South
5,West
6,West
7,West
8,West
9,West


In [ ]:
tablasmicro = tablasmicro.withColumn("Region", F.regexp_replace(F.col("Region"), 'Centarl', 'Central'))

In [ ]:
tablasmicro.select("Region").limit(20).toPandas()

,Region
0,South
1,South
2,West
3,South
4,South
5,West
6,West
7,West
8,West
9,West




Convertimos las columnas de fechas `Account_Opening_Date` y `Product_Contracting_Date` a tipo date

In [ ]:
from pyspark.sql.functions import to_date

for i in ["Account_Opening_Date", "Product_Contracting_Date"]:
    tablasmicro = tablasmicro.withColumn(i, to_date(F.col(i), 'dd/MM/yyyy'))

In [ ]:
tablasmicro.select("Account_Opening_Date", "Product_Contracting_Date").limit(20).toPandas()

,Account_Opening_Date,Product_Contracting_Date
0,2016-11-08,2016-11-11
1,2016-11-08,2016-11-11
2,2016-06-12,2016-06-16
3,2015-10-11,2015-10-18
4,2015-10-11,2015-10-18
5,2014-06-09,2014-06-14
6,2014-06-09,2014-06-14
7,2014-06-09,2014-06-14
8,2014-06-09,2014-06-14
9,2014-06-09,2014-06-14




Sustituir el carácter "-" por "_" en el nombre de la columna `Sub-Product`

In [ ]:
tablasmicro = tablasmicro.withColumnRenamed("Sub-Product", "Sub_Product")



Eliminamos por último las columnas `_c0` y `Row_ID`, ya que no aportan información

In [ ]:
tablasmicro = tablasmicro.drop(*["_c0", "Row_ID"])



Comprobamos cómo han quedado nuestros datos

In [ ]:
tablasmicro.limit(5).toPandas()

,Order_ID,Account_Opening_Date,Product_Contracting_Date,Contracting_Mode,Customer_Name,Client,Country,Postal_Code,Region,Product_ID,Product,Sub_Product,Sales,Quantity,Discount,Profit,Customer_num,City,State
0,CA-2016-152156,2016-11-08,2016-11-11,Web,Claire Gute,Final Client,United States,42420,South,FUR-BO-10001798,Office product,Pension Plan,261.959991,2.0,0.00,41.913601,12520,Henderson,Kentucky
1,CA-2016-152156,2016-11-08,2016-11-11,Web,Claire Gute,Final Client,United States,42420,South,FUR-CH-10000454,Office product,Paysheets,731.940002,3.0,0.00,219.582001,12520,Henderson,Kentucky
2,CA-2016-138688,2016-06-12,2016-06-16,Web,Darrin Van Huff,Big Accounts,United States,90036,West,OFF-LA-10000240,Loans and mortgages,Mastercard,14.620000,2.0,0.00,6.871400,13045,Los Angeles,California
3,US-2015-108966,2015-10-11,2015-10-18,Bank Branch,Sean O'Donnell,Final Client,United States,33311,South,FUR-TA-10000577,Office product,Home insurance,957.577515,5.0,0.45,-383.031006,20335,Fort Lauderdale,Florida
4,US-2015-108966,2015-10-11,2015-10-18,Bank Branch,Sean O'Donnell,Final Client,United States,33311,South,OFF-ST-10000760,Loans and mortgages,Others,22.368000,2.0,0.20,2.516400,20335,Fort Lauderdale,Florida


In [ ]:
tablasmicro.dtypes

[('Order_ID', 'string'),
 ('Account_Opening_Date', 'date'),
 ('Product_Contracting_Date', 'date'),
 ('Contracting_Mode', 'string'),
 ('Customer_Name', 'string'),
 ('Client', 'string'),
 ('Country', 'string'),
 ('Postal_Code', 'int'),
 ('Region', 'string'),
 ('Product_ID', 'string'),
 ('Product', 'string'),
 ('Sub_Product', 'string'),
 ('Sales', 'float'),
 ('Quantity', 'float'),
 ('Discount', 'float'),
 ('Profit', 'float'),
 ('Customer_num', 'int'),
 ('City', 'string'),
 ('State', 'string')]



### Escritura del fichero en parquet



Guardamos el fichero con todas las modificaciones en un parquet en la sandbox

In [ ]:
#path del futuro fichero parquet (también en el sandbox)
parquet_path = DATA_PATH + "/bank_products_bbva2019.parquet"
#Escribimos el nuevo fichero parquet
tablasmicro.write.mode("overwrite").parquet(parquet_path)



## Crear la tabla en CrossData 



Este paso es **IMPRESCINDIBLE**. Si no se publica en Crossdata, no se pueden atacar los datos desde ningún BI.

Creamos la base de datos si no existe

In [ ]:
curso = 'dspb_2019'

In [ ]:
xd.sql("CREATE DATABASE IF NOT EXISTS " + curso)

DataFrame[]



Creamos la tabla a partir del fichero parquet que acabamos de guardar en la Sandbox

In [ ]:
xd.sql("CREATE TABLE IF NOT EXISTS " + curso + ".bank_products_bbva2019 USING PARQUET OPTIONS (path'" + parquet_path + "')")

DataFrame[]



Especificamos la base de datos que queremos utilizar (para no tener que escribirla cada vez que hagamos una query) y mostramos las tablas que hay en ella

In [ ]:
xd.sql("USE " + curso)
xd.sql("show tables").show()

+------------------+--------------------+-----------+
|          database|           tableName|isTemporary|
+------------------+--------------------+-----------+
|prueba_curso_micro|bank_products_bbv...|      false|
+------------------+--------------------+-----------+





Comprobamos que la tabla se ha generado correctamente y que los tipos de datos de las columnas son los correctos

In [ ]:
xd.sql("SELECT * FROM " + curso + ".bank_products_bbva2019")

DataFrame[Order_ID: string, Account_Opening_Date: date, Product_Contracting_Date: date, Contracting_Mode: string, Customer_Name: string, Client: string, Country: string, Postal_Code: int, Region: string, Product_ID: string, Product: string, Sub_Product: string, Sales: float, Quantity: float, Discount: float, Profit: float, Customer_num: int, City: string, State: string]

In [ ]:
xd.sql("SELECT * FROM " + curso + ".bank_products_bbva2019 LIMIT 1").toPandas()

,Order_ID,Account_Opening_Date,Product_Contracting_Date,Contracting_Mode,Customer_Name,Client,Country,Postal_Code,Region,Product_ID,Product,Sub_Product,Sales,Quantity,Discount,Profit,Customer_num,City,State
0,CA-2016-152156,2016-11-08,2016-11-11,Web,Claire Gute,Final Client,United States,42420,South,FUR-BO-10001798,Office product,Pension Plan,261.959991,2.0,0.0,41.913601,12520,Henderson,Kentucky


In [ ]:
xd.sql("DESCRIBE " + curso + ".bank_products_bbva2019").toPandas()

,col_name,data_type,comment
0,Order_ID,string,None
1,Account_Opening_Date,date,None
2,Product_Contracting_Date,date,None
3,Contracting_Mode,string,None
4,Customer_Name,string,None
5,Client,string,None
6,Country,string,None
7,Postal_Code,int,None
8,Region,string,None
9,Product_ID,string,None
